Mount GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Setup working directory

In [ ]:
import os
# set base dir
BASE_DIR='main_proj_v1' 
os.environ['BASE_DIR'] = 'main_proj_v1'

!git init .
!git remote add -t * -f origin https://github.com/rdj0im/rl-atsc.git
!git checkout master

!mkdir -p $BASE_DIR
!mkdir -p  /content/gdrive/MyDrive/models
!mkdir -p  /content/gdrive/MyDrive/models/outputs
os.chdir(BASE_DIR)
# get git repo
!git init .
!git remote add -t * -f origin https://github.com/rdj0im/rl-atsc.git
!git checkout main
# set up credentials
!sed -i '/\[remote \"origin\"\]/Q' .git/config
!cat /content/gdrive/MyDrive/Colab\ Notebooks/config >> .git/config

Install SUMO

In [ ]:
!sudo add-apt-repository -y ppa:sumo/stable
!sudo apt-get update
!sudo apt-get -y install sumo sumo-tools sumo-doc


Set sumo env vars

In [ ]:
# Set environment variable
os.environ['SUMO_HOME'] = '/usr/share/sumo'
os.environ['LIBSUMO_AS_TRACI'] = '1' #Optional: for a huge performance boost (~8x) with Libsumo (No GUI)

**SUMO RL**

Install modules

In [ ]:
# !pip install 'setuptools<67'
# !pip install gym==0.21.0
# !pip install stable_baselines3
# !pip install git+https://github.com/DLR-RM/stable-baselines3.git
# !pip install sumo-rl
# !pip install -U ray[rllib] tensorflow torchh
!pip install torch

Main



In [ ]:
import os
import sys
import sumo_rl

if "SUMO_HOME" in os.environ:
    tools = os.path.join(os.environ["SUMO_HOME"], "tools")
    sys.path.append(tools)
else:
    sys.exit("Please declare the environment variable 'SUMO_HOME'")
import traci

import sumo_rl


def obs_fun_wave_wait():

if __name__ == "__main__":
    env = sumo_rl.parallel_env(
        net_file="sumo_files/4x4_grid_network.net.xml",
        route_file="sumo_files/4x4_grid_routes.rou.xml",
        out_csv_name="outputs/ppo",
        single_agent=False,
        use_gui=True,
        num_seconds=100000,
    )
    env.reset(seed=33)
    s=False
    while env.agents:
    # this is where you would insert your policy
        actions = {agent: env.action_space(agent).sample() for agent in env.agents}  
        action_spaces=env.action_spaces
        a_s={agent: env.action_space(agent) for agent in env.agents}
        observations, rewards, terminations, truncations, infos = env.step(actions)
    env.close()

**------------------------Code Ends Here----------------------**

In [ ]:
import os
import sys
import sumo_rl
# import gym
# from stable_baselines3 import PPO

if "SUMO_HOME" in os.environ:
    tools = os.path.join(os.environ["SUMO_HOME"], "tools")
    sys.path.append(tools)
else:
    sys.exit("Please declare the environment variable 'SUMO_HOME'")
import traci

from sumo_rl import SumoEnvironment


if __name__ == "__main__":
    env = sumo_rl.environment.env.SumoEnvironment(
        net_file="sumo_files/4x4_grid_network.net.xml",
        route_file="sumo_files/4x4_grid_routes.rou.xml",
        out_csv_name="/content/gdrive/MyDrive/models/outputs/ppo",
        single_agent=True,
        use_gui=False,
        num_seconds=100000,
    )

    model = PPO(
        env=env,
        policy="MlpPolicy",
        learning_rate=0.001,
        n_steps=15,
        gamma=0.99,
        batch_size=20,
        verbose=1,
        seed=9,
    )
    model.learn(total_timesteps=100000)
    model.save('/content/gdrive/MyDrive/models/ppo_4x4_sing')

In [ ]:
import os
import sys
import numpy as np
from gym import spaces
from ray import tune
import sumo_rl
from ray.rllib.algorithms.a3c import A3CConfig

config = A3CConfig() 

config = config.training(lr=0.01, grad_clip=30.0) 

config = config.resources(num_gpus=0) 

config = config.rollouts(num_rollout_workers=4) 

config = config.environment("CartPole-v1") 

print(config.to_dict())  

# Build a Algorithm object from the config and run 1 training iteration.

algo = config.build()  

algo.train()  


if "SUMO_HOME" in os.environ:
    tools = os.path.join(os.environ["SUMO_HOME"], "tools")
    sys.path.append(tools)
else:
    sys.exit("Please declare the environment variable 'SUMO_HOME'")


if __name__ == "__main__":
    ray.init()
    env=sumo_rl.parallel_env(
        net_file="sumo_files/4x4_grid_network.net.xml",
        route_file="sumo_files/4x4_grid_routes.rou.xml",
        out_csv_name="/content/gdrive/MyDrive/models/outputs/ppo",
        use_gui=False,
        num_seconds=100000,)
    trainer = A3CTrainer(
        env="4x4grid",
        config={
            "multiagent": {
                "policies": {"0": (A3CTFPolicy, spaces.Box(low=np.zeros(11), high=np.ones(11)), spaces.Discrete(2), {})},
                "policy_mapping_fn": (lambda id: "0"),  # Traffic lights are always controlled by this policy
            },
            "lr": 0.001,
            "no_done_at_end": True,
        },
    )
    while True:
        print(trainer.train())  # distributed training step


In [ ]:
!python --version


---

**Save, Commit and Push**

In [ ]:
# import time
# from IPython.display import display, Javascript
# import hashlib

# # trigger save of current notebook 

# def save_notebook(file_path):
#     start_md5 = hashlib.md5(open(file_path,'rb').read()).hexdigest()
#     display(Javascript('IPython.notebook.save_checkpoint();'))
#     current_md5 = start_md5

#     while start_md5 == current_md5:
#         time.sleep(1)
#         current_md5 = hashlib.md5(open(file_path,'rb').read()).hexdigest()

# copy saved nb to current dir
# file_path='/content/gdrive/MyDrive/Colab Notebooks/main_proj_v1.ipynb'
# save_notebook(file_path)

# 🥲
# ''' Trigger a Save using Ctrl+S beforehand '''
# 🥲

!cp /content/gdrive/MyDrive/Colab\ Notebooks/main_proj_v1.ipynb main_proj_v1.ipynb 
# push to remote
!git add -A
!git commit -m "ipynb save triggered from nb"
!git push
